# Problem 3

Before you start to work on Problem 3, please visit [this page](https://canvas.mit.edu/files/1128379/download?download_frd=1) to download the installation instructions.

Please install Julia (v1.5.3), JuMP (v0.20.0) and Gurobi (v9.1.1) on your computer as instructed by the tutorial on Canvas. These are the versions that the teaching team will provide support for. You should use Gurobi as the solver for all JuMP related problems, and use the Plots or StatsPlots packages for graphing.

Before you submit this Jupyter notebook, please use (1) `Kernel` $\rightarrow$ `Restart & Run All`, and (2) `File` $\rightarrow$ `Download a` $\rightarrow$ `HMTL (.html)` on the notebook menu to make the output of your codes available to the TAs.



## Class Assignments

The Salanter Akiba Riverdale (SAR) Academy is a coeducational, private Modern Orthodox Jewish day school located in New York City. Ever summer, the SAR Academy must create class assignments for their elementary school students. Each grade of 80-100 students must be divided into different classes. Requests for assignments are made by parents, teachers, and school therapists. These requests include pairs of students that should be placed together, pairs of students that should not be placed together, and requests for students to be placed in classes that better suit their unique or special academic needs. These requests often conflict with each other, and it falls on the administration to prioritize which requests should be fulfilled over others.

In this exercise, we will solve a simplified version of the problem faced by the SAR Academy with 40 students and two classes. (The full optimization problem is currently being used to assist administrators at the SAR Academy.) The parents or guardians of each of the 40 students are asked to submit preferences for class 1 or class 2. These preferences often depend on the teaching style of the teachers, the teachers older siblings have had in the past, and characteristics of the class (one class is called an "inclusion class," which is better for students with academic needs). The parents give a ranking of 1 to the class they prefer (their first choice), and a ranking of 2 to their second choice. This data, as well as the gender of each of the students, is given in the spreadsheet `ClassAssignments.csv`.   When solving this problem, please use the prepared Jupyter notebook.

## Part (a)

The  problem  faced  by  the  SAR  Academy  is  to  decide  which  students  should  be assigned to which classes,  to satisfy as many of the parent preferences as possible (note thatsince  smaller  numbers  for  the  preferences  are  better,  this  is  a  minimization  problem).   Each student must be assigned to exactly one class, and there should be exactly 20 students in eachclass.

### (i)

Formulate this problem as an integer optimization problem. Describe the decision variables, objective function, and constraints.

On your PDF writeup, write down the formulation of this problem. Be sure to clearly indicate what your decision variables, objective, and constraints are.


Let's work on some Julia code now. First, import the dataset from the spreadsheet `ClassAssignments.csv`. You can do it in the cell below.

In [1]:
## if you have not installed package "CSV", please uncomment the line below and execute it
# using Pkg; Pkg.add("CSV")
using DataFrames, CSV
portfolio = CSV.read("ClassAssignments.csv", DataFrame)

,Student Number,Parent Preference for Class 1,Parent Preference for Class 2,Male or Female? (M or F)
,Int64,Int64,Int64,String
1,1,1,2,M
2,2,1,2,M
3,3,2,1,M
4,4,1,2,M
5,5,1,2,M
6,6,2,1,M
7,7,1,2,M
8,8,2,1,M
9,9,1,2,M


Then, in the cell below, write Julia/JuMP codes to describe the program.

In [2]:
#--- Model specification
using JuMP, DataFrames, Gurobi

# Defining the model
model = Model(with_optimizer(Gurobi.Optimizer))

# Gathering all of the preferences into one single list from the Portfolio DataFrame
class_1_p = portfolio[:,2]
class_2_p = portfolio[:,3]

all_p_values = append!(class_1_p, class_2_p)

# Decision variables are x[1]...x[80] in order to represent the 2 classes with 40 students each
@variable(model, x[1:80] >= 0)

# Minimum Objective Function
@objective(model, Min, sum(all_p_values[j]*x[j] for j in 1:80))

    
# Each student must be assigned to exactly one class,
@constraint(model, con[i = 1:40], x[i] + x[i+40] ==  1)


# There should be exactly 20 students in the first class
@constraint(model,sum(x[i] for i in 1:40) == 20)


# There should be exactly 20 students in the second class
@constraint(model,sum(x[i+40] for i in 1:40) == 20)

# Take a look at the model
model

Academic license - for non-commercial use only - expires 2021-04-24


A JuMP Model
Minimization problem with:
Variables: 80
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 42 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 80 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: con, x

In the cell below, write Julia/JuMP codes to solve this problem with Gurobi.

In [3]:
#--- Write codes here to solve this problem with Gurobi

# Optimizing the model
optimize!(model)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 42 rows, 80 columns and 160 nonzeros
Model fingerprint: 0x3bcc4627
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 42 rows and 80 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2000000e+01   0.000000e+00   2.000000e+00      0s
Extra simplex iterations after uncrush: 1
       1    4.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  4.200000000e+01

User-callback calls 36, time in user-callback 0.00 sec


What is the optimal solution?  Give the values of the decision variables as well as the value of the optimal objective function value in the below cell.

In [4]:
#--- Write codes here to print your solutions

# Showing the optimal solutions
@show value.(x)

# Showing the optimal objective value
@show objective_value(model)

value.(x) = [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
objective_value(model) = 42.0


42.0

### (iii)

How  many  students  received  their  first  choice  class,  according  to  the  parent preferences?   (HINT:  You  should  be  able  to  answer  this  by  just  looking  at  the  optimal objective function value.) Write your solution in the Markdown box below.

38 students received their first choice class, according to the parent preferences.



P * 1 + (40-P) * 2 = 42 ==> Solve for P

P = 38, which will represent the number of students who received their first choice class. The 1 and the 2 is coming from the ranking of 1 and 2 and then the P and 40 - P is just enumrating those.

## Part (b)

After looking at the optimal solution, the SAR academy decided that they would like to adjust the formulation for a better boy/ girl ratio in the classes. They would like to limit the boys in each class to no more than 12.

### (i)

What constraint(s) do you need to add to your model to incorporate this adjustment? Write down your solution on the PDF writeup.

### (ii)

Add the necessary constraints and resolve your model. In the cell below, write Julia/JuMP codes to solve this problem with Gurobi.

In [5]:
#--- Write codes here to solve this problem with Gurobi

using JuMP, DataFrames, Gurobi

# Defining the model
model = Model(with_optimizer(Gurobi.Optimizer))

# Gathering all of the preferences into one single list from the DataFrame
class_1_p = portfolio[:,2]
class_2_p = portfolio[:,3]

all_p_values = append!(class_1_p, class_2_p)

# Decision variables
@variable(model, x[1:80] >= 0)

# Objective Function
@objective(model, Min, sum(all_p_values[j]*x[j] for j in 1:80))

    
# Each student must be assigned to exactly one class,
@constraint(model, con[i = 1:40], x[i] + x[i+40] ==  1)


# There should be exactly 20 students in the first class
@constraint(model,sum(x[i] for i in 1:40) == 20)


# There should be exactly 20 students in the second class
@constraint(model,sum(x[i+40] for i in 1:40) == 20)

# The boys in the first class should be limited to no more than 12
@constraint(model,sum(x[i] for i in 1:23) <= 12)

# The boys in the second class should be limited to no more than 12
@constraint(model,sum(x[i+40] for i in 1:23) <= 12)

# Optimizing the model
optimize!(model)

Academic license - for non-commercial use only - expires 2021-04-24
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 44 rows, 80 columns and 206 nonzeros
Model fingerprint: 0x4ca290b6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 44 rows and 80 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+01   0.000000e+00   8.000000e+00      0s
Extra simplex iterations after uncrush: 2
       2    4.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  4.600000000e+01

User-callback calls 42, time in user-callback 0.00 sec


What is the optimal solution now? Give the values of the decision variables as well as the value of the optimal objective function value in the below cell.

In [6]:
#--- Write codes here to print your solutions

# Showing the optimal solutions
@show value.(x)

# Showing the optimal objective value
@show objective_value(model)

value.(x) = [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0]
objective_value(model) = 46.0


46.0

How does it compare to the previous solution? Discuss this in the Markdown box below.

The optimal objective value of the previous solution was 42 while this optimal objective value is 46, meaning our objective value got worse. This makes intuitive and mathematical sense as we have added constraints of the boys in each class being no more than 12. The solution now has to satisfy these constraints, meaning the solution has gotten worse as there is less flexbility as the problem has "narrowed"/"became tighter."

### (iii)

How many students received their first choice class in this new solution, according to the parent preferences? Write down your solution in the Markdown box below.

34 students received their first choice class, according to the parent preferences.



P * 1 + (40-P) * 2 = 46 ==> Solve for P

P = 34, which will represent the number of students who received their first choice class. The 1 and the 2 is coming from the ranking of 1 and 2 and then the P and 40 - P is just enumrating those.


This number is less than the number from the previous solution, which makes sense since we have added constraints here, which would make the number of students who receiived their first choice less as more constraints need to be satisfied. 

## Part (c)

Now, we will add some logical constraints to capture additional
preferences of parents, teachers, and school therapists.

### (i)

Students 10 and 11 are twins, and the school has a policy that twins must be placed in different classes. What constraint(s) needs to be added to the model to implement this policy? Write down your solution on the PDF writeup.

### (ii)

Students 4, 9, 15, 25, 30, and 36 are all from the same neighborhood. The school would like to put at least 2 students from this neighborhood in each class. What constraint(s) needs to be added to the model to implement this policy? Write down your solution on the PDF writeup.

### (iii)

The school therapist strongly recommends that students 20 and 21 are placed in the same classroom, that student 1 is placed in classroom 2, and that student 40 is placed in classroom 2. What constraint(s) needs to be added to the model to implement this policy? Write down your solution on the PDF writeup.

### (iv)

Add all of these constraints to your model, and solve it again.

In the cell below, write Julia/JuMP codes to solve this new problem. Please use Gurobi as the solver.

In [7]:
#--- Write Julia/JuMP codes to solve this new problem with Gurob

using JuMP, DataFrames, Gurobi

# Defining the model
model = Model(with_optimizer(Gurobi.Optimizer))

# Gathering all of the preferences into one single list from the DataFrame
class_1_p = portfolio[:,2]
class_2_p = portfolio[:,3]

all_p_values = append!(class_1_p, class_2_p)

# Decision variables
@variable(model, x[1:80] >= 0)

# Objective Function
@objective(model, Min, sum(all_p_values[j]*x[j] for j in 1:80))

    
# Each student must be assigned to exactly one class,
@constraint(model, con[i = 1:40], x[i] + x[i+40] ==  1)


# There should be exactly 20 students in the first class
@constraint(model,sum(x[i] for i in 1:40) == 20)


# There should be exactly 20 students in the second class
@constraint(model,sum(x[i+40] for i in 1:40) == 20)

# The boys in the first class should be limited to no more than 12
@constraint(model,sum(x[i] for i in 1:23) <= 12)

# The boys in the second class should be limited to no more than 12
@constraint(model,sum(x[i+40] for i in 1:23) <= 12)

# The school has a policy that twins, students 10 and 11 namely, must be placed in different classes
@constraint(model,x[10] + x[11] == 1)
@constraint(model,x[50] + x[51] == 1)

# The school would like to put at least 2 students from this neighborhood in each class
@constraint(model,x[4] + x[9] + x[15] + x[25] + x[30] + x[36] >= 2)
@constraint(model,x[44] + x[49] + x[55] + x[65] + x[70] + x[76] >= 2)

# The school therapist strongly recommends that students 20 and 21 are placed in the same classroom
@constraint(model,x[20] == x[21])
@constraint(model,x[60] == x[61])

# Student 1 needs to be placed in classroom 2
@constraint(model,x[41] == 1)

# Student 40 needs to be placed in classroom 2
@constraint(model,x[80] == 1)

# Optimizing the model
optimize!(model)

Academic license - for non-commercial use only - expires 2021-04-24
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 52 rows, 80 columns and 228 nonzeros
Model fingerprint: 0x7d6d8374
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 49 rows and 72 columns
Presolve time: 0.00s
Presolved: 3 rows, 10 columns, 17 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1000000e+01   4.000000e+00   0.000000e+00      0s
       3    4.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds
Optimal objective  4.600000000e+01

User-callback calls 42, time in user-callback 0.00 sec


What is the optimal solution now? Give the values of the decision variables as well as the value of the optimal objective function value in the below cell.

In [8]:
#--- Write codes here to print your solutions

# Showing the optimal solutions
@show value.(x)

# Showing the optimal objective value
@show objective_value(model)

value.(x) = [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0]
objective_value(model) = 46.0


46.0

How does the objective value compare to the previous solution, without all of these constraints? What does this tell us? Discuss this in the Markdown box below.

*Hooray! You're answering the last problem of the pset here :)*

The optimal objective value to the previous solution was 46 and the optimal objective value to this solution is also 46, meaning the optimal objective value DID NOT CHANGE despite adding several constraints relating to Students 20,21,1,and 40. This tells us that these extra constraints are not quite as helpful for us as there seem to be many solutions that yield this same optimal objective value of 46. In other words, adding extra constraints was not particularly useful as the optimal objective value seems to be more rigid and not sensitive as it stayed the same.